In [ ]:
%matplotlib inline
import pandas as pd
import ast
ted = pd.read_csv("ted_main.csv")

In [ ]:
ted.head()

In [ ]:
ted.describe()

In [ ]:
ted["views"].mean()

In [ ]:
ted.corr()

In [ ]:
# Total count for each rating. Most talks pick up the inspiring rating.

import operator
ratings = {}
for index, rating_str in ted["ratings"].iteritems():
    ratings_list = ast.literal_eval(rating_str)
    for rating in ratings_list:
        ratings[rating["name"]] = ratings.get(rating["name"], 0) + rating["count"]
for i in sorted(ratings.items(), key=operator.itemgetter(1)):
    print(i)

In [ ]:
# Adding columns with counts for all ratings for a talk.

ratings_df = ted.copy()
ratings_list = ratings.keys()
for rating in ratings_list:
    ratings_df[rating] = 0

In [ ]:
# Speakers with more than 1 talk.

import operator
name_dict = {}
for name in ted["main_speaker"].iteritems():
    name_dict[name[1]] = name_dict.get(name[1], 0) + 1
count = 0
for k in sorted(name_dict.items(), key = operator.itemgetter(1)):
    if k[1] > 1:
        print ("%s - %s" % (k[0], k[1]))

In [ ]:
# Set of all tags

tag_set = set()
for tag_str in ted["tags"]:
    tag_list = ast.literal_eval(tag_str)
    for tag in tag_list:
        tag_set.add(tag)
print(tag_set)

In [ ]:
# Print tags per talk of speaker.

def print_tags_for_speaker(speaker):
    rosling = ted[ted["main_speaker"] == speaker]
    tag_set = []
    for tag_str in rosling["tags"]:
        tag_list = ast.literal_eval(tag_str)
        tag_set.append(tag_list)
    print(tag_set)

print_tags_for_speaker("Hans Rosling")
# print_tags_for_speaker("Juan Enriquez")

In [ ]:
# Most viewed talks.

most_viewed = ted[["title", "main_speaker", "views"]].sort_values("views", ascending=False)
most_viewed.head()

In [ ]:
# Most commented on talks

most_commented = ted[["title", "main_speaker", "views", "comments"]].sort_values("comments", ascending=False)
most_commented.head()

In [ ]:
# There doesn't seem to be any correlation between views and comments. The top viewed TED talk is not the top commented 
# inspite of having 10 times more views than the top commented one "Militant Atheism".

ted.plot(x = "views", y = "comments", kind = "scatter")

In [ ]:
# We observe that views and languages are slightly positively correlated 0.3, TED talks with more than 10 million 
# have atleast 28 languages

ted.plot(x = "views", y = "languages", kind = "scatter")
ted[ted["views"] > 10000000].languages.sort_values().head(1)

In [ ]:
# Each rating with the associated score for each talk normalized over the number of views. 

def populate_ratings():
    for index, rating_str in ratings_df["ratings"].iteritems():
        max_rating = -1
        ratings_list = ast.literal_eval(rating_str)
        for rating in ratings_list:
            ratings_df.loc[index, rating["name"]] = rating["count"] / ted.iloc[index]["views"]
populate_ratings()

In [ ]:
display_list = ["title", "main_speaker"] + list(ratings_list)
ratings_df[display_list].head()

In [ ]:
from IPython.display import display
for rating in ratings_list:
    display(ratings_df.sort_values(by = rating, ascending = False)[["title", rating, "views"]].head(5))
    
# This gives us a more accurate description of whether the talk was funny/inspiring etc. This is per user how many 
# people found it funny as opposed to overall coz it may be biased for a talk with more views.

In [ ]:
ratings_df[list(ratings_list)].corr()

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(ratings_df[list(ratings_list)], alpha=0.2, figsize=(14, 14), diagonal='kde')

# We were expecting positive correlation between some ratings which we were able to verify. 
# Didn't seem to find any negative correlation which was surprising.